In [1]:
import paragami

import autograd
from autograd import numpy as np
from autograd import scipy as sp

# Use the original scipy for functions we don't need to differentiate.
import scipy as osp

# Using `paragami` with a positive semi-definite matrix parameter.

We will begin by considering a simple symmetric positive semi-definite matrix.  In general, we can write:

$$
A = \left[
\begin{matrix}
a_{11} & a_{12} & a_{13}  \\
a_{21} & a_{22} & a_{23}  \\
a_{31} & a_{32} & a_{33}  \\
\end{matrix}
\right]
$$

...although, of course, symmetry and positive semi-definiteness impose constraints on the entries $a_{ij}$ of $A$ which we will discuss below.

## Flattening and folding.

Let us first consider how to represent $A$ as a vector, and then as an unconstrained vector.

Because $A$ is symmetric, $a_{21} = a_{12}$, $a_{31} = a_{13}$, and $a_{23} = a_{32}$.  This means that to reproduce $A$ we only need to store the bold values:

$$
A = \left[
\begin{matrix}
\mathbf{a_{11}} & \mathbf{a_{12}} & \mathbf{a_{13}}  \\
a_{21} & \mathbf{a_{22}} & \mathbf{a_{23}}  \\
a_{31} & a_{32} & \mathbf{a_{33}}  \\
\end{matrix}
\right]
$$

If we wanted to represent $A$ as an flat array, we could write

$$
A_{flat} = \left[
\begin{matrix}
\mathbf{a_{11}} \\
\mathbf{a_{12}} \\
\mathbf{a_{22}} \\
\mathbf{a_{13}} \\
\mathbf{a_{23}} \\
\mathbf{a_{33}} \\
\end{matrix}
\right]
$$

and fully recover $A$ from $A_{flat}$.

Converting to and from $A$ and $A_{flat}$ can be done with the `flatten` method of a `paragami.PSDMatrixPattern` pattern.  

For the moment, because we are simply re-stacking the values of $A$ into a vector, not transforming into an unconstrained space, we use the option `free=False`.  We will discuss the `free=True` option shortly.

In [2]:
# A sample positive semi-definite matrix.
a = np.eye(3) + np.random.random((3, 3))
a = 0.5 * (a + a.T)

# Define a pattern and fold.
a_pattern = paragami.PSDMatrixPattern(size=3)
a_flat = a_pattern.flatten(a, free=False)

print('Now, a_flat contains the elements of a exactly as shown in the formula above.\n')
print('a:\n{}\n'.format(a))
print('a_flat:\n{}\n'.format(a_flat))

Now, a_flat contains the elements of a exactly as shown in the formula above.

a:
[[1.43029514 0.91026842 0.28169565]
 [0.91026842 1.96292882 0.67954451]
 [0.28169565 0.67954451 1.39334116]]

a_flat:
[1.43029514 0.91026842 1.96292882 0.28169565 0.67954451 1.39334116]



We can also convert from $A_{flat}$ back to $A$ by 'folding'.

In [3]:
print('Folding the flattened value recovers the original matrix.\n')
a_fold = a_pattern.fold(a_flat, free=False)
print('a:\n{}\n'.format(a))
print('a_fold:\n{}\n'.format(a_flat))

Folding the flattened value recovers the original matrix.

a:
[[1.43029514 0.91026842 0.28169565]
 [0.91026842 1.96292882 0.67954451]
 [0.28169565 0.67954451 1.39334116]]

a_fold:
[1.43029514 0.91026842 1.96292882 0.28169565 0.67954451 1.39334116]



## Free flattening and folding.

Not every length-six vector represents a valid positive semi-definite matrix.  If the attribute `validate` of a `paragami.PSDMatrixPattern` is true, then folding automatically checks for validity.

In [4]:
print('The diagonal of a positive semi-definite matrix must not be less',
      'than 0, and folding checks this when validate=True, which it is by default:\n')
a_flat_bad = np.array([-1, 0, 0, 0, 0, 0])
print('A bad folded value: {}'.format(a_flat_bad))
try:
    a_fold_bad = a_pattern.fold(a_flat_bad, free=False)
except ValueError as err:
    print('Folding with a_pattern raised the following ValueError:\n{}'.format(err))

print('\nIf validate is false, folding will produce an invalid matrix without an error:\n')
a_pattern.validate = False
a_fold_bad = a_pattern.fold(a_flat_bad, free=False)
print('Folding a non-pd matrix with validate=False:\n{}'.format(a_fold_bad))

print('\nHowever, it will not produce a matrix of the wrong shape even when validate is False:\n')
a_flat_very_bad = np.array([1, 0, 0])
print('A very bad folded value: {}.'.format(a_flat_very_bad))
try:
    a_fold_very_bad = a_pattern.fold(a_flat_very_bad, free=False)
except ValueError as err:
    print('Folding with a_pattern raised the following ValueError:\n{}'.format(err))

# Let's set validate back to true.
a_pattern.validate = True

The diagonal of a positive semi-definite matrix must not be less than 0, and folding checks this when validate=True, which it is by default:

A bad folded value: [-1  0  0  0  0  0]
Folding with a_pattern raised the following ValueError:
Diagonal is less than the lower bound 0.0.

If validate is false, folding will produce an invalid matrix without an error:

Folding a non-pd matrix with validate=False:
[[-1.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]

However, it will not produce a matrix of the wrong shape even when validate is False:

A very bad folded value: [1 0 0].
Folding with a_pattern raised the following ValueError:
Wrong length for PDMatrix flat value.


Sometimes (e.g. when performing optimization) it is convenient to have a flattened value that can legally take any value of the correct length.  For a positive semi-definite matrix, this can be achieved by a Cholesky decomposition.  The details aren't important here, as `paragami` takes care of the transformation behind the scenes with the option `free=True`. 

In [5]:
print('The free flat value a_freeflat is not immediately recognizable as a.\n')
a_freeflat = a_pattern.flatten(a, free=True)
print('a:\n{}\n'.format(a))
print('a_freeflat:\n{}\n'.format(a_freeflat))

print('However, it transforms correctly back to a when folded.\n')
a_freefold = a_pattern.fold(a_freeflat, free=True)
print('a_fold:\n{}\n'.format(a_freefold))

print('Any length-six vector will free fold back to a valid positive',
      'semi-definite matrix up to floating point error.',
      'Let\'s draw 100 random vectors, fold them, and check that this is true.\n')
# Draw random free vectors and confirm that they are positive semi definite.
def assert_is_pd(mat):
    eigvals = np.linalg.eigvals(mat)
    assert np.min(eigvals) >= -1e-8
for raw in range(100):
    a_rand_freeflat = np.random.normal(scale=2, size=(6, ))
    a_rand_fold = a_pattern.fold(a_rand_freeflat, free=True)
    assert_is_pd(a_rand_fold)


print('Note that you will get an incorrect values or errors if you mix',
      'free and non-free folding and flattening!',
      '\nIn general, validate cannot be relied upon to catch such errors.\n')
a_pattern.validate = False
a_fold_free_unfree_mix = a_pattern.fold(a_freeflat, free=False)
print('a_fold_free_unfree_mix:\n{}\n'.format(a_fold_bad))



The free flat value a_freeflat is not immediately recognizable as a.

a:
[[1.43029514 0.91026842 0.28169565]
 [0.91026842 1.96292882 0.67954451]
 [0.28169565 0.67954451 1.39334116]]

a_freeflat:
[0.17894041 0.76112615 0.16235011 0.23554143 0.42529939 0.07290737]

However, it transforms correctly back to a when folded.

a_fold:
[[1.43029514 0.91026842 0.28169565]
 [0.91026842 1.96292882 0.67954451]
 [0.28169565 0.67954451 1.39334116]]

Any length-six vector will free fold back to a valid positive semi-definite matrix up to floating point error. Let's draw 100 random vectors, fold them, and check that this is true.

Note that you will get an incorrect values or errors if you mix free and non-free folding and flattening! 
In general, validate cannot be relied upon to catch such errors.

a_fold_free_unfree_mix:
[[-1.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]



# Using flattening and folding for optimization.

Suppose we are interested in optimizing some function of $A$, say, a normal model in which the data $x_n \sim \mathcal{N}(0, A)$.  Specifically, Let the data be $X = \left(x_1, ..., x_N\right)$, where $x_n \in \mathbb{R}^3$, and write a loss function as

$$
\ell\left(X, A\right) =
    -\sum_{n=1}^N \log P(x_n | A) =
    \frac{1}{2}\sum_{n=1}^N \left(x_n^T A^{-1} x_n - \log|A|\right) 
$$

Let's simulate some data under this model.

In [24]:
np.random.seed(42)

N = 1000

# True value of A
true_a = np.eye(3) * np.diag(np.array([1, 2, 3])) + np.random.random((3, 3)) * 0.1
true_a = 0.5 * (true_a + true_a.T)

# Data
def draw_data(N, true_a):
    return np.random.multivariate_normal(
        mean=np.zeros(3), cov=true_a, size=(N, ))

x = draw_data(N, true_a)
print('X shape: {}'.format(x.shape))

# Later it will be convenient to separate the loss into a vector of
# per-observation losses and a regularization term.
def get_loss_by_n(x, a):
    a_inv = np.linalg.inv(a)
    a_det_sign, a_log_det = np.linalg.slogdet(a)
    #assert a_det_sign > 0
    return 0.5 * (np.einsum('ni,ij,nj->n', x, a_inv, x) + a_log_det)

def get_loss_regularization(lam, a):
    a_det_sign, a_log_det = np.linalg.slogdet(a)
    #assert a_det_sign > 0
    return  lam * a_log_det

def get_loss(x, lam, a):
    loss_by_n = get_loss_by_n(x, a)
    return np.sum(loss_by_n) + get_loss_regularization(lam, a)

print('Loss at true parameter: {}'.format(get_loss(x, lam, true_a)))

X shape: (1000, 3)
Loss at true parameter: 2394.604318307261


We would like to minimize the function `loss` using tools like `scipy.optimize.minimize`.  Standard optimization functions take vectors, not matrices, as input, and often require the vector to take valid values in the entire domain.

As-written, our loss function takes a positive definite matrix as an input.  We can wrap the loss as a funciton of the free flattened value using the `paragami.FlattenedFunction` class.  The resulting function can be passed directly to `autograd` and `scipy.optimize`.

In [28]:
get_freeflat_loss = paragami.FlattenedFunction(
    original_fun=get_loss, patterns=a_pattern, free=True, argnums=2)

print('The two losses are the same when evalated on the folded and flat values:\n')
print('Original loss:\t\t{}'.format(get_loss(x, lam, true_a)))
true_a_freeflat = a_pattern.flatten(true_a, free=True)
print('Free-flattened loss: \t{}'.format(get_freeflat_loss(x, lam, true_a_freeflat)))

print('\nNow, use the flattened function to optimize with autograd.\n')

get_freeflat_loss_grad = autograd.grad(get_freeflat_loss, argnum=2)
get_freeflat_loss_hessian = autograd.hessian(get_freeflat_loss, argnum=2)

def get_optimum(x, lam):
    loss_opt = osp.optimize.minimize(
        method='trust-ncg',
        x0=np.zeros(a_pattern.flat_length(free=True)),
        fun=lambda par: get_freeflat_loss(x, lam, par),
        jac=lambda par: get_freeflat_loss_grad(x, lam, par),
        hess=lambda par: get_freeflat_loss_hessian(x, lam, par),
        options={'gtol': 1e-8, 'disp': True})
    return loss_opt

loss_opt = get_optimum(x, lam)
print('Optimization successful: {}\nOptimal value: {}'.format(
    loss_opt.success, loss_opt.fun))

The two losses are the same when evalated on the folded and flat values:

Original loss:		2394.604318307261
Free-flattened loss: 	2394.604318307261

Now, use the flattened function to optimize with autograd.

         Current function value: 2392.424627
         Iterations: 9
         Function evaluations: 11
         Gradient evaluations: 10
         Hessian evaluations: 10
Optimization successful: False
Optimal value: 2392.4246267108656


The optimization was in the free flattened space, so to get the optimal value of $A$ we must fold it.

In [30]:
print('True a:\n{}\nOptimal a:\n{}'.format(
    true_a,
    a_pattern.fold(loss_opt.x, free=True)))

True a:
[[1.03745401 0.07746864 0.03950388]
 [0.07746864 2.01560186 0.05110853]
 [0.03950388 0.05110853 3.0601115 ]]
Optimal a:
[[ 1.0666994   0.07756251  0.04867278]
 [ 0.07756251  1.89015495 -0.03092313]
 [ 0.04867278 -0.03092313  2.93888267]]


Suppose we wanted to use the sandwich estimator 